In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import collections

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.set_printoptions(edgeitems=2)
torch.manual_seed(123)


In [7]:
from torchvision import datasets, transforms
data_path = '../data-unversioned/p1ch6/'
cifar10 = datasets.CIFAR10(
    data_path, train=True, download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))

cifar10_val = datasets.CIFAR10(
    data_path, train=False, download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))

label_map = {0: 0, 2: 1}
cifar2 = [(img, label_map[label])for img, label in cifar10 if label in [0, 2]]
cifar2_val = [(img, label_map[label])for img, label in cifar10_val if label in [0, 2]]

Files already downloaded and verified
Files already downloaded and verified


In [4]:
conv = nn.Conv2d(3, 16, kernel_size=3) # 
conv.weight.shape , conv.bias.shape  #weight = output*input*3,3

(torch.Size([16, 3, 3, 3]), torch.Size([16]))

In [8]:
img, _ = cifar2[0]
output = conv(img.unsqueeze(0))
img.unsqueeze(0).shape, output.shape

(torch.Size([1, 3, 32, 32]), torch.Size([1, 16, 30, 30]))

In [9]:
conv = nn.Conv2d(3, 1, kernel_size=3, padding=1) # 커널3,3 패딩 1 적용
output = conv(img.unsqueeze(0))
img.unsqueeze(0).shape, output.shape

(torch.Size([1, 3, 32, 32]), torch.Size([1, 1, 32, 32]))

In [10]:
pool = nn.MaxPool2d(2)
output = pool(img.unsqueeze(0))
img.unsqueeze(0).shape, output.shape

(torch.Size([1, 3, 32, 32]), torch.Size([1, 3, 16, 16]))

In [21]:
img.shape

torch.Size([3, 32, 32])

In [19]:
img.unsqueeze(0).view(-1,8*8*8).shape

torch.Size([6, 512])

In [11]:
model = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, padding=1), # 16개 32*32 (16*32*#32)
            nn.Tanh(),
            nn.MaxPool2d(2), # 16개 16*16 (16*16*16)
            nn.Conv2d(16, 8, kernel_size=3, padding=1), # 8개 16*16 (8*16*16)
            nn.Tanh(),
            nn.MaxPool2d(2), # 8개 8*8 (8*8*8)
            # 중간에 차원을 바꿔주는것이 필요하다. 하지만 nn.Sequential을 사용하면 중간에 끼우는거 불가능
            nn.Linear(8 * 8 * 8, 32),
            nn.Tanh(),
            nn.Linear(32, 2))

In [31]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1) # input : (3,32,32) / output : (16,32,32)
        self.act1 = nn.Tanh()
        self.pool1 = nn.MaxPool2d(2) # output : 16개 16*16 (16*16*16)
        self.conv2 = nn.Conv2d(16, 8, kernel_size=3, padding=1) # output : 8개 16*16 (8*16*16)
        self.act2 = nn.Tanh()
        self.pool2 = nn.MaxPool2d(2) # 8개 8*8 (8*8*8)
        self.fc1 = nn.Linear(8 * 8 * 8, 32)
        self.act3 = nn.Tanh()
        self.fc2 = nn.Linear(32, 2)
        
        
    def forward(self,x):
        out = self.pool1(self.act1(self.conv1(x)))
        out = self.pool2(self.act2(self.conv2(out)))
        out = out.view(-1,8*8*8)
        out = self.act3(self.fc1(out))
        out = self.fc2(out)
        return out 
    
    
# 컨볼루션 채널수가 점점 감소하고 선형계층으로 낮은 더 낮은 차원을 출력한다.


In [33]:
model = Net()
model

Net(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act1): Tanh()
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(16, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act2): Tanh()
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=512, out_features=32, bias=True)
  (act3): Tanh()
  (fc2): Linear(in_features=32, out_features=2, bias=True)
)

In [37]:
numel_list = [p.numel() for p in model.parameters()]
print('전체 파라미터 수 : ',sum(numel_list))
print('각 층별 파라미터 수 : ',numel_list)

전체 파라미터 수 :  18090
각 층별 파라미터 수 :  [432, 16, 1152, 8, 16384, 32, 64, 2]


In [38]:
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1) 
        self.conv2 = nn.Conv2d(16, 8, kernel_size=3, padding=1) 
        self.fc1 = nn.Linear(8 * 8 * 8, 32)
        self.fc2 = nn.Linear(32, 2)
        
        
    def forward(self,x):
        out = F.max_pool2d(torch.tanh(self.conv1(x)),2)
        out = F.max_pool2d(torch.tanh(self.conv2(out)),2)
        out = out.view(-1,8*8*8)
        out = torch.tanh(self.fc1(out))
        out = self.fc2(out)
        return out 
    
    


In [41]:
model = Net()
model(img.unsqueeze(0))

tensor([[-0.1006,  0.0149]], grad_fn=<AddmmBackward0>)

In [43]:
import datetime

def training_loop(n_epochs, optimizer, model, loss_fn, train_loader):
    for epoch in range(1, n_epochs + 1): 
        loss_train = 0.0
        for imgs, labels in train_loader:  # 데이터 로거가 만들어준 배치 안에서 데이터셋 순회
            
            outputs = model(imgs)  # 모델에 배치를 넣어줌 
            
            loss = loss_fn(outputs, labels)  # 손실값 계산

            optimizer.zero_grad()  # 이전 기울기 지움
            
            loss.backward()  # 역전파(신경망이 학습할 모든 파라미터에 대한 기울기 계산)
             
            optimizer.step() # 모델 업데이트

            loss_train += loss.item()  # 에포크동안 손실값을 모두 더함 

        if epoch == 1 or epoch % 10 == 0:
            print('{} Epoch {}, Training loss {}'.format(datetime.datetime.now(), epoch,loss_train / len(train_loader))) # 배치단위 평균 손실값

In [45]:
train_loader = torch.utils.data.DataLoader(cifar2,batch_size=64,shuffle=True)
model = Net()
optimizer = optim.SGD(model.parameters(),lr=1e-2) # 옵티마이저 
loss_fn = nn.CrossEntropyLoss() # 크로스엔트로피 loss

training_loop(n_epochs=100,
              optimizer=optimizer,
              model=model,
              loss_fn=loss_fn,
              train_loader=train_loader)


2024-03-29 10:02:15.752595 Epoch 1, Training loss 0.588388922677678
2024-03-29 10:02:49.553147 Epoch 10, Training loss 0.35369018270711233
2024-03-29 10:03:29.130675 Epoch 20, Training loss 0.31357031557590337
2024-03-29 10:04:07.567162 Epoch 30, Training loss 0.27930168968856717
2024-03-29 10:04:41.413289 Epoch 40, Training loss 0.25484782628193026
2024-03-29 10:05:15.080960 Epoch 50, Training loss 0.23337636551090107
2024-03-29 10:05:49.234828 Epoch 60, Training loss 0.21509803371254804
2024-03-29 10:06:22.541276 Epoch 70, Training loss 0.198244459974538
2024-03-29 10:07:13.784843 Epoch 80, Training loss 0.18209621248541363
2024-03-29 10:07:51.653327 Epoch 90, Training loss 0.16750281992231963
2024-03-29 10:08:27.269004 Epoch 100, Training loss 0.1553064205559196


In [46]:
# 평가
train_loader = torch.utils.data.DataLoader(cifar2, batch_size=64,shuffle=False)
val_loader = torch.utils.data.DataLoader(cifar2_val, batch_size=64,shuffle=False)

def validate(model, train_loader, val_loader):
    for name, loader in [("train", train_loader), ("val", val_loader)]:
        correct = 0
        total = 0
        with torch.no_grad():  # 파라미터 업데이트를 하지 않을것이므로 기울기 필요 x 
            for imgs, labels in loader:
                outputs = model(imgs)
                _, predicted = torch.max(outputs, dim=1) # 가장 높은값을 가진 인덱스를 출력 
                total += labels.shape[0]  # 에제수를 세어서 total을 배치 크기만큼 증가 
                correct += int((predicted == labels).sum())  # 예측이랑 실제값이랑 얼마나 맞았는지 세어 합친다 

        print("Accuracy {}: {:.2f}".format(name , correct / total))

validate(model, train_loader, val_loader)

Accuracy train: 0.94
Accuracy val: 0.90


In [1]:
# 저장
torch.save(model.state_dict(), 'deepmodel.pt')

NameError: name 'torch' is not defined